In [ ]:
!pip install pyspark py4j

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro").getOrCreate()

# Nova seção

In [ ]:
df_nomes = spark.read.csv("/nomes_aleatorios.txt")

In [ ]:
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



In [ ]:
df_nomes = df_nomes.withColumnRenamed(df_nomes.columns[0], "Nomes")

In [ ]:
df_nomes.show(10)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



In [ ]:
df_nomes = df_nomes.withColumn("Escolaridade", F.when(F.rand() < 1/3, "Fundamental")
                                               .when(F.rand() < 2/3, "Medio")
                                               .otherwise("Superior"))
df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|    Superior|
|    Jamie Russell| Fundamental|
|   Edward Kistler| Fundamental|
|    Sheila Maurer|       Medio|
| Donald Golightly| Fundamental|
|       David Gray|       Medio|
|      Joy Bennett|    Superior|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|       Medio|
|    Brian Farrell|       Medio|
+-----------------+------------+
only showing top 10 rows



In [ ]:
df_nomes = df_nomes.withColumn("Pais", F.when(F.rand() < 1/13, "Argentina")
                                        .when(F.rand() < 2/13, "Bolivia")
                                        .when(F.rand() < 3/13, "Brasil")
                                        .when(F.rand() < 4/13, "Chile")
                                        .when(F.rand() < 5/13, "Colombia")
                                        .when(F.rand() < 6/13, "Equador")
                                        .when(F.rand() < 7/13, "Guiana")
                                        .when(F.rand() < 8/13, "Guiana Francesa")
                                        .when(F.rand() < 9/13, "Paraguai")
                                        .when(F.rand() < 10/13, "Peru")
                                        .when(F.rand() < 11/13, "Suriname")
                                        .when(F.rand() < 12/13, "Uruguai")
                                        .otherwise("Venezuela")
                                        )
df_nomes.show(10)

+-----------------+------------+---------------+
|            Nomes|Escolaridade|           Pais|
+-----------------+------------+---------------+
|   Frances Bennet|    Superior|         Brasil|
|    Jamie Russell| Fundamental|       Colombia|
|   Edward Kistler| Fundamental|      Argentina|
|    Sheila Maurer|       Medio|         Brasil|
| Donald Golightly| Fundamental|        Bolivia|
|       David Gray|       Medio|        Equador|
|      Joy Bennett|    Superior|Guiana Francesa|
|      Paul Kriese| Fundamental|         Guiana|
|Berniece Ornellas|       Medio|         Brasil|
|    Brian Farrell|       Medio|          Chile|
+-----------------+------------+---------------+
only showing top 10 rows



In [ ]:
df_nomes = df_nomes.withColumn("AnoNascimento", (1945 + (F.rand() * 66)).cast("int"))
df_nomes.show(10)

+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|   Frances Bennet|    Superior|         Brasil|         1955|
|    Jamie Russell| Fundamental|       Colombia|         2009|
|   Edward Kistler| Fundamental|      Argentina|         2004|
|    Sheila Maurer|       Medio|         Brasil|         1961|
| Donald Golightly| Fundamental|        Bolivia|         2006|
|       David Gray|       Medio|        Equador|         1974|
|      Joy Bennett|    Superior|Guiana Francesa|         1966|
|      Paul Kriese| Fundamental|         Guiana|         2000|
|Berniece Ornellas|       Medio|         Brasil|         1950|
|    Brian Farrell|       Medio|          Chile|         1979|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



In [ ]:
df_select = df_nomes.select("*").where(df_nomes.AnoNascimento > 2000)
df_select.show(10)

+-----------------+
|            Nomes|
+-----------------+
|    Jamie Russell|
|   Edward Kistler|
| Donald Golightly|
|         Mary Lee|
|   Wilfredo Grant|
|  Kenneth Rayburn|
|    Michael Agnew|
|     James Barton|
|      Milton Rowe|
|Marguerite Rankin|
+-----------------+
only showing top 10 rows



In [ ]:
df_nomes.createOrReplaceTempView("pessoas")
query = """
    SELECT Nomes
    FROM pessoas
    WHERE AnoNascimento > 2000
"""
df_sql = spark.sql(query)
df_sql.show(10)

+-----------------+
|            Nomes|
+-----------------+
|    Jamie Russell|
|   Edward Kistler|
| Donald Golightly|
|         Mary Lee|
|   Wilfredo Grant|
|  Kenneth Rayburn|
|    Michael Agnew|
|     James Barton|
|      Milton Rowe|
|Marguerite Rankin|
+-----------------+
only showing top 10 rows



In [ ]:
millenials = df_nomes.select("*").where((F.col("AnoNascimento") > 1980) & (F.col("AnoNascimento") < 1994)).count()
print(millenials)

1970126


In [ ]:
query = """
    SELECT count(Nomes) as ContagemMillenials
    FROM pessoas
    WHERE AnoNascimento > 1980 AND AnoNascimento < 1994
"""
contagem_sql = spark.sql(query)
contagem_sql.show()

+------------------+
|ContagemMillenials|
+------------------+
|           1970126|
+------------------+



In [ ]:
query = """
    SELECT Pais,
           CASE WHEN AnoNascimento BETWEEN 1944 AND 1964 THEN 'Baby Boomers'
                WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 'Geração X'
                WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 'Millennials'
                ELSE 'Geração Z'
                END AS Geracao,
           COUNT(Nomes) AS Quantidade
    FROM pessoas
    GROUP BY Pais, Geracao
    ORDER BY Pais, Geracao, Quantidade
"""

df_resultado = spark.sql(query)

df_resultado.show(52)

+---------------+------------+----------+
|           Pais|     Geracao|Quantidade|
+---------------+------------+----------+
|      Argentina|Baby Boomers|    232696|
|      Argentina|   Geração X|    174456|
|      Argentina|   Geração Z|    187137|
|      Argentina| Millennials|    174974|
|        Bolivia|Baby Boomers|    429881|
|        Bolivia|   Geração X|    322327|
|        Bolivia|   Geração Z|    343637|
|        Bolivia| Millennials|    322794|
|         Brasil|Baby Boomers|    546374|
|         Brasil|   Geração X|    410689|
|         Brasil|   Geração Z|    436922|
|         Brasil| Millennials|    409930|
|          Chile|Baby Boomers|    559755|
|          Chile|   Geração X|    419884|
|          Chile|   Geração Z|    448125|
|          Chile| Millennials|    419380|
|       Colombia|Baby Boomers|    483464|
|       Colombia|   Geração X|    364079|
|       Colombia|   Geração Z|    387514|
|       Colombia| Millennials|    364061|
|        Equador|Baby Boomers|    